# Projeto Aprendizado de Máquina

## CMC-13 Introdução a Ciência de Dados
## Equipe:
### André Luiz de Melo Thiessen
### Nikollas da Silva Antes
### Pedro Anacleto Martins Senna de Oliveira

## Importando bibliotecas

In [232]:
#Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

## Importando os datasets

In [233]:
#Importando as bases de dados
movies = pd.read_csv("movies.csv", on_bad_lines='skip', sep = ';')
ratings = pd.read_csv("ratings.csv", sep = ';')
users = pd.read_csv("users.csv")

## Data Cleaning
- Primeiramente, foi verificado a existência de valores nulos (dados faltantes)

In [234]:
# Verificando a existência de valores nulos (dados faltantes)
print(movies.isnull().values.any())
print(ratings.isnull().values.any())
print(users.isnull().values.any())

False
False
False


- Depois foram apagadas as colunas que julgou-se impertinentes para o aprendizado em questão
    - Title, Timestamp, Zip-code e name: irrelevantes
    - Genres dos movies: Deixaria a árvore muito grande, optamos por retirar tal atributo

In [235]:
#Apagando as colunas irrelevantes
movies = movies.drop(columns='Title')
ratings = ratings.drop(columns='Timestamp')
users = users.drop(columns='Zip-code')
users = users.drop(columns='name')
movies = movies.drop(columns='Genres')

## Transformações de Dados

- Foi alterado a data de nascimento (birthday) para o padrão proposto no lab, agrupando em idades

In [236]:
format_string = "%m/%d/%Y"
def calculateAge(birthDate): 
    today = date.today() 
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))
    return age

for i in users.index:
    [month, day, year] = users['birthday'][i].split('/')
    if(day == '0'):
        users.loc[i, 'birthday'] = month+'/'+'1/'+year
    date = datetime.strptime(users['birthday'][i], format_string)
    age = calculateAge(date)
    users.loc[i, 'birthday'] = age
    
    #Agrupando os usuários em 
for i in users.index:
    if(int(users['birthday'][i])<18):
        users.loc[i, 'birthday'] = '1'
    elif (int(users['birthday'][i])<24):
        users.loc[i, 'birthday'] = '18'
    elif (int(users['birthday'][i])<35):
        users.loc[i, 'birthday'] = '25'
    elif (int(users['birthday'][i])<45):
        users.loc[i, 'birthday'] = '35'
    elif (int(users['birthday'][i])<50):
        users.loc[i, 'birthday'] = '45'
    elif (int(users['birthday'][i])<55):
        users.loc[i, 'birthday'] = '50'
    else:
        users.loc[i, 'birthday'] = '56'


- Além disso, fez-se um merge das tabelas para facilitar o trabalho futuro
- Nomeou-se o novo dataframe de df

In [237]:
df = pd.merge(ratings,users)
df = pd.merge(df,movies)
df = df.drop(columns='UserID')
display(df)

,MovieID,Rating,Gender,Occupation,birthday
0,1193,5,F,10,1
1,1193,5,M,16,56
2,1193,4,M,12,25
3,1193,4,M,7,25
4,1193,5,M,1,50
...,...,...,...,...,...
1000176,2198,5,M,17,1
1000177,2703,3,M,14,35
1000178,2845,1,M,17,18
1000179,3607,5,F,20,18


## Árvore de decisão

### Cálculo da entropia

In [238]:
def entropy(df):
    rating = df.columns[1]
    values, counts = np.unique(df[rating], return_counts = True)
    prob = counts/(counts.sum())
    entropy = sum(-prob*np.log2(prob))
    return entropy

entropy(df)

2.1002336337757876

### Cálculo do ganho de informação

In [259]:
def ganho_info(df, atributo):
    values, counts = np.unique(df[atributo], return_counts=True)
    prob = counts/counts.sum()
    entropy2 = 0
    for i in range(len(values)):
        cases = df.loc[df[atributo] == values[i]]
        entropy2 += counts[i]*entropy(cases)/counts.sum()
    ganho = entropy(df) - entropy2
#     print(ganho)
    return ganho, atributo
    
ganho_info(df, 'birthday')
        

(0.0036821434675720077, 'birthday')

### Escolha dos melhores atributos

In [260]:
def melhor_atributo(df):
    order = []
    for column in df.columns:
        if(column != 'Rating' and column != 'MovieID'):
#             print(ganho_info(df, column))
            order.append(ganho_info(df, column))
    order.sort(reverse=True)
    print(order)
    
melhor_atributo(df)

[(0.0038236898645949147, 'Occupation'), (0.0036821434675720077, 'birthday'), (0.0003299270469576676, 'Gender')]


## Algoritmo ID3

- Utiliza dos atributos com melhor ganho de informação para ficarem acima na árvore

## Média Truncada

In [241]:
#Cálculo da Média truncada
i = 0
soma = 0
while i < len(ratings):
    soma =ratings.Rating[i] + soma
    i = i + 1
#Calcula a média das classficações sem considerar nenhum outro fator externo
Média = soma/len(ratings)
#Arredonda para o valor inteiro mais próximo para condizer com os tipos de classificações do filme
truncada = round(Média)

print("A média truncada é:",truncada)

A média truncada é: 4


## Conclusão

O trabalho solicitado foi bastante condizente com a teoria apresentada em sala de aula, propiciando a aplicação dos conceitos teóricos básicos, essenciais para o desenvolvimento da matéira de ciência de dados.
A execução do trabalho foi complexa, não pela dificuldade de entender o embasamento teórico que permeia os conceitos aplicados, mas sim pela dificuldade de aplicar os algoritmos necessários na linguagem computacional.
O grupo gestaria de deixar como sugestão desenvolver, seja em sala de aula, por meio de vídeo-aulas, ou ainda por meio de materiais escritos no google classroom, a parte prática da disciplina, mostrando como aplicar os conceitos na linguagem computacional, conforme é exigido nas práticas de laboratório.

## Implementação

A implementação do trabalho foi realizada em python e a IDE utilizada foi o Jupyter Notebook.